Should we calculate difference? https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/

Convert wind dirs to degrees for numerical values? https://study.com/cimages/multimages/16/cb_1_copy.jpeg
    - No: convert to direction vector: https://www.tensorflow.org/tutorials/structured_data/time_series
    
Major issue to resolve - data is not strictly time series as it contains the data from many sites over time, with overlapping time frames. How to prevent model from considering one site's time series data as an extension of the previous site's?

In [ ]:
import datetime
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('training_data/training_set.csv')

In [ ]:
# this does not really work accurately
locs = pd.read_csv('locations.csv')
bbox = (89.868, 176.001, 2.592, -47.309)
# bbox = (locs.lon.min(), locs.lon.max(), locs.lat.min(), locs.lat.max())

m = plt.imread('map.png')

fig, ax = plt.subplots(figsize=(20, 20))

ax.scatter(locs.lon, locs.lat, zorder=1, alpha=0.2, c='b', s=10)
ax.set_xlim(bbox[0], bbox[1])
ax.set_ylim(bbox[2], bbox[3])
ax.invert_yaxis()
ax.imshow(m, zorder=0, extent=bbox, aspect='equal', origin='lower')

locs.lon.min(), locs.lon.max(), locs.lat.min(), locs.lat.max()

In [ ]:
data['Location'].value_counts()

In [ ]:
(data.isna().sum() / data.shape[0]) * 100 # what percent of each column is NA?
data["RainTomorrow"].value_counts() # our data is not balanced between classes

In [ ]:
# convert to binary
data["RainTomorrow"].replace(('Yes', 'No'), (1, 0), inplace=True)
data["RainToday"].replace(('Yes', 'No'), (1, 0), inplace=True)
data.drop(columns=['Location', 'Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm', 'RISK_MM'], inplace=True)

# convert direction strings to degrees
data['WindGustDir'].replace(
    ('N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW','SW','WSW','W','WNW','NW','NNW'),
    (0, 22.5, 45, 67.5, 90, 112.5, 135, 157.5, 180, 202.5, 225, 247.5, 270, 292.5, 315, 337.5),
    inplace=True
)

data['WindDir9am'].replace(
    ('N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW','SW','WSW','W','WNW','NW','NNW'),
    (0, 22.5, 45, 67.5, 90, 112.5, 135, 157.5, 180, 202.5, 225, 247.5, 270, 292.5, 315, 337.5),
    inplace=True
)

data['WindDir3pm'].replace(
    ('N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW','SW','WSW','W','WNW','NW','NNW'),
    (0, 22.5, 45, 67.5, 90, 112.5, 135, 157.5, 180, 202.5, 225, 247.5, 270, 292.5, 315, 337.5),
    inplace=True
)

In [ ]:
# drop na in listed columns
data.dropna(
    axis=0, how='any',
    subset=['WindGustDir','WindDir9am','WindDir3pm','WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm'],
    inplace=True
)

data.dropna(axis=0, how='any', inplace=True)

data_date = pd.to_datetime(data.pop('Date'))

In [ ]:
# plot features over time
plot_cols = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed']
# plot_cols = ['Rainfall']
plot_features = data[plot_cols]
plot_features.index = data_date
plot_features.plot(subplots=True, figsize=(10,10))

In [ ]:
data.describe().transpose()

In [ ]:
# plot distribution of wind data
plt.hist2d(data['WindGustDir'], data['WindGustSpeed'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind Direction [deg]')
plt.ylabel('Wind Velocity [m/s]')

In [ ]:
# convert all wind data to vectors
wv = data.pop('WindGustSpeed')
wd = data.pop('WindGustDir')*np.pi/180
data['WindGustX'] = wv*np.cos(wd)
data['WindGustY'] = wv*np.sin(wd)

wv = data.pop('WindSpeed9am')
wd = data.pop('WindDir9am')*np.pi/180
data['Wind9amX'] = wv*np.cos(wd)
data['Wind9amY'] = wv*np.sin(wd)

wv = data.pop('WindSpeed3pm')
wd = data.pop('WindDir3pm')*np.pi/180
data['Wind3pmX'] = wv*np.cos(wd)
data['Wind3pmY'] = wv*np.sin(wd)

In [ ]:
# plot distribution of new wind vectors
plt.hist2d(data['WindGustX'], data['WindGustY'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind Direction [deg]')
plt.ylabel('Wind Velocity [m/s]')
ax = plt.gca()
ax.axis('tight')

In [ ]:
timestamp_s = data_date.map(datetime.datetime.timestamp)
day = 24*60*60
year = (365.2425)*day
data['daysin'] = np.sin(timestamp_s * (2 * np.pi/day))
data['daycos'] = np.cos(timestamp_s * (2 * np.pi/day))
data['yearsin'] = np.sin(timestamp_s * (2 * np.pi/year))
data['yearcos'] = np.cos(timestamp_s * (2 * np.pi/year))

In [ ]:
# split and normalize all the data
n = len(data)
train_df = data[0:int(n*0.7)]
val_df = data[int(n*0.7):int(n*0.9)]
test_df = data[int(n*0.9):]

cols_to_norm = list(data.columns)
cols_to_norm.remove('RainToday')
cols_to_norm.remove('RainTomorrow')

train_mean = train_df[cols_to_norm].mean()
train_std = train_df[cols_to_norm].std()

train_df.loc[:,cols_to_norm] = (train_df.loc[:,cols_to_norm] - train_mean) / train_std
val_df.loc[:,cols_to_norm] = (val_df.loc[:,cols_to_norm] - train_mean) / train_std
test_df.loc[:,cols_to_norm] = (test_df.loc[:,cols_to_norm] - train_mean) / train_std

In [ ]:
data_std = (data - train_mean) / train_std
data_melted = data_std.melt(var_name='Column', value_name='Normalized')
var_rows = int(data_melted.shape[0]/21) # number of rows for each variable

plt.figure(figsize=(12,6))
ax = sns.violinplot(x='Column', y='Normalized', data=data_melted[:var_rows*6])
_ = ax.set_xticklabels(data.keys()[:6], rotation=90)

plt.figure(figsize=(12,6))
ax = sns.violinplot(x='Column', y='Normalized', data=data_melted[var_rows*6:var_rows*11])
_ = ax.set_xticklabels(data.keys()[6:11], rotation=90)

plt.figure(figsize=(12,6))
ax = sns.violinplot(x='Column', y='Normalized', data=data_melted[var_rows*11:var_rows*16])
_ = ax.set_xticklabels(data.keys()[11:16], rotation=90)

plt.figure(figsize=(12,6))
ax = sns.violinplot(x='Column', y='Normalized', data=data_melted[var_rows*16:var_rows*21])
_ = ax.set_xticklabels(data.keys()[16:21], rotation=90)

In [ ]:
# https://www.tensorflow.org/tutorials/structured_data/time_series#1_indexes_and_offsets
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None, batch_size=32):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns],axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    def plot(self, model=None, plot_col='RainTomorrow', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        predictions = 0
        for n in range(max_n):
            plt.subplot(3, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()

        plt.xlabel('Time [h]')
        
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=batch_size,)

        ds = ds.map(self.split_window)

        return ds
    
    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

In [ ]:
w = WindowGenerator(input_width = 7, label_width=1, shift=1, label_columns=['RainTomorrow'])
w.plot()

In [ ]:
for example_inputs, example_labels in w.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
epochs = 20

linear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])

val_performance = {}
performance = {}

def compile_and_fit(model, window, lr=0.001, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    model.compile(loss=tf.losses.BinaryCrossentropy(), optimizer=tf.optimizers.Adam(lr=lr),
        metrics=[tf.metrics.BinaryAccuracy()])
#     model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(),
#         metrics=[tf.metrics.MeanAbsoluteError()])
    
#     model.compile(loss='binary_crossentropy', optimizer='adam')
    
    history = model.fit(window.train, epochs=epochs, validation_data=window.val, callbacks=[early_stopping])
    return history    

In [ ]:
history = compile_and_fit(linear_model, w)

val_performance['Linear'] = linear_model.evaluate(w.val)
performance['Linear'] = linear_model.evaluate(w.test, verbose=0)

In [ ]:
dense_model = keras.Sequential([
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid')
])

history = compile_and_fit(dense_model, w)
val_performance['Dense'] = dense_model.evaluate(w.val)
performance['Dense'] = dense_model.evaluate(w.test, verbose=0)

In [ ]:
# preds = w.plot(dense_model)
# print(preds)

preds = dense_model.predict_classes(w.train_df)


In [ ]:
tmp = w.train_df.reset_index()['RainTomorrow']
count = 0

# for i in range(100,200):
#     print(preds[i][0])

for i in range(1,preds.shape[0]):
# for i in range(100,200):
#     print(preds[i][0])
#     print(tmp[i])
    if ((preds[i][0]-tmp[i])**2) == 0:
        count+=1
print((preds[1][0] - tmp[1])^2)  
count/preds.shape[0]

In [ ]:
w = WindowGenerator(input_width = 7, label_width=1, shift=1, label_columns=['RainTomorrow'])
multi_dense_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=32, activation='relu'),
    keras.layers.Dense(units=32, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid'),
    keras.layers.Reshape([1, -1])
])

history = compile_and_fit(multi_dense_model, w)
val_performance['Multi step dense'] = multi_dense_model.evaluate(w.val)
performance['Multi step dense'] = multi_dense_model.evaluate(w.test, verbose=0)

In [ ]:
c_width = 7
w = WindowGenerator(input_width = c_width, label_width=1, shift=1, label_columns=['RainTomorrow'])
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=c_width, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

history = compile_and_fit(cnn_model, w)
val_performance['Conv'] = cnn_model.evaluate(w.val)

w.plot(cnn_model)
inputs = w.train
preds = cnn_model(inputs)
preds

In [ ]:


--------------------------------------



In [ ]:
n = len(data)
train_df = data[0:int(n*0.7)]
val_df = data[int(n*0.7):int(n*0.9)]
test_df = data[int(n*0.9):]

In [ ]:
epochs = 5
lr=0.000001
batch_size = 32

w = WindowGenerator(input_width = 30, label_width=30, shift=1, label_columns=['RainTomorrow'])

lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(16, return_sequences=True),
#     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(32, return_sequences=True),
#     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64, return_sequences=True),
#     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

history = compile_and_fit(lstm_model, w, lr=lr)
val_performance['lstm_model'] = lstm_model.evaluate(w.val)
performance['lstm_model'] = lstm_model.evaluate(w.test, verbose=0)


In [ ]:
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

history.history